In [1]:
# Enable autoreload in Jupyter
%load_ext autoreload
%autoreload 2

**USDA API Food Data Processing** <br>
Getting nutrient data for available branded foods

In [2]:
import requests
import dotenv
import pandas as pd

url = "https://api.nal.usda.gov/fdc/v1/foods/list"
_api_key = dotenv.get_key("credentials.env", "USDA_FOOD_KEY")

In [3]:
# get list of branded / basic foundational foods, sorted by name
response = requests.get(url, params={
    "api_key":_api_key,
    "dataType":["Foundation"],
    "sortBy":"dataType.keyword"
})

if response.status_code == 200:
    foods_data = response.json()
    foods_df = pd.DataFrame(foods_data)
    print(f"Total records: {len(foods_df)}")
else:
    print(f"Error: {response.status_code} - {response.text}")



Total records: 50


In [4]:
foods_df

,fdcId,description,dataType,publicationDate,ndbNumber,foodNutrients
0,2262074,"Almond butter, creamy",Foundation,2022-04-28,12195,"[{'number': '717', 'name': 'Daidzin', 'amount'..."
1,2257045,"Almond milk, unsweetened, plain, refrigerated",Foundation,2022-04-28,100276,"[{'number': '404', 'name': 'Thiamin', 'amount'..."
2,1999631,"Almond milk, unsweetened, plain, shelf stable",Foundation,2021-10-28,14091,"[{'number': '631', 'name': 'PUFA 22:5 n-3 (DPA..."
3,2003590,"Apple juice, with added vitamin C, from concen...",Foundation,2021-10-28,9400,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
4,1750340,"Apples, fuji, with skin, raw",Foundation,2020-10-30,9504,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
5,1750341,"Apples, gala, with skin, raw",Foundation,2020-10-30,9503,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
6,1750342,"Apples, granny smith, with skin, raw",Foundation,2020-10-30,9502,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
7,1750343,"Apples, honeycrisp, with skin, raw",Foundation,2020-10-30,9501,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
8,1750339,"Apples, red delicious, with skin, raw",Foundation,2020-10-30,9500,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
9,2346414,"Applesauce, unsweetened, with added vitamin C",Foundation,2022-10-28,9401,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."


In [5]:
rows, cols = foods_df.shape
print(f"Number of rows: {rows}")
print(f"Number of columns: {cols}")
print(foods_df.columns)

Number of rows: 50
Number of columns: 6
Index(['fdcId', 'description', 'dataType', 'publicationDate', 'ndbNumber',
       'foodNutrients'],
      dtype='object')


# CSV Import
Data is split across dozens of csv files. Need to join.

In [6]:
import importlib, food_db_client
importlib.reload(food_db_client)
import re
from food_db_client import FoodDBClient

def filter_rows_by_substrings(df, substrings):
    if isinstance(substrings, str):
        substrings = [substrings]

    pattern = '|'.join([re.escape(s) for s in substrings])
    return df[df['description'].str.contains(pattern, case=False, na=False)]

# Example usage:
filtered_df = filter_rows_by_substrings(foods_df, ["protein", "carb", "sugar", "fat"])
print(filtered_df.shape)
filtered_df

(6, 6)


,fdcId,description,dataType,publicationDate,ndbNumber,foodNutrients
37,2644289,"Beans, kidney, dark red, canned, sodium added,...",Foundation,2023-10-26,100318,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
38,2644290,"Beans, kidney, light red, canned, sodium added...",Foundation,2023-10-26,100319,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
45,2514744,"Beef, ground, 80% lean meat / 20% fat, raw",Foundation,2023-04-20,23572,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
46,2514743,"Beef, ground, 90% lean meat / 10% fat, raw",Foundation,2023-04-20,23562,"[{'number': '303', 'name': 'Iron, Fe', 'amount..."
47,746758,"Beef, loin, tenderloin roast, separable lean o...",Foundation,2019-12-16,23377,"[{'number': '204', 'name': 'Total lipid (fat)'..."
48,746759,"Beef, loin, top loin steak, boneless, lip-on, ...",Foundation,2019-12-16,23385,"[{'number': '406', 'name': 'Niacin', 'amount':..."


In [7]:
first_fdc_id = filtered_df.iloc[0]['fdcId']

FoodDBClient.get_food_by_id(first_fdc_id)

,fdcId,description,publicationDate,foodNutrients,dataType,foodClass,inputFoods,foodComponents,foodAttributes,nutrientConversionFactors,ndbNumber,isHistoricalReference,foodCategory.id,foodCategory.code,foodCategory.description
0,2644289,"Beans, kidney, dark red, canned, sodium added,...",10/26/2023,"[{'nutrient': {'id': 2045, 'number': '951', 'n...",Foundation,FinalFood,"[{'id': 123937, 'foodDescription': 'beans, kid...",[],[],"[{'id': 23042, 'value': 6.25, 'type': '.Protei...",100318,False,16,1600,Legumes and Legume Products


In [8]:
all_nutrients = FoodDBClient.get_all_nutrients()
all_nutrients

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0
...,...,...,...,...,...
472,2064,Oligosaccharides,MG,NaN,2250.0
473,2065,Low Molecular Weight Dietary Fiber (LMWDF),G,293.4,1306.0
474,2068,Vitamin E,MG,959.0,7810.0
475,2067,Vitamin A,UG,960.0,7430.0


In [9]:
food_nutrients = FoodDBClient.get_food_nutrients(first_fdc_id)
print(food_nutrients.shape)
food_nutrients = food_nutrients.drop(columns=["id","derivation_id","footnote", "min_year_acquired"])
food_nutrients

(20, 11)


c:\Users\Natha\source\repos\Code Projects\food\FoodOptimizer\Backend\food_db_client.py:78: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  nutrients_df = pd.read_csv(filePath)


,fdc_id,nutrient_id,amount,data_points,min,max,median
136351,2644289,1071,2.00000,8.0,2.000,2.0000,2.0000
136352,2644289,1093,172.30000,8.0,94.700,268.0000,153.5000
136353,2644289,1009,12.16000,8.0,11.430,13.2300,12.0700
136354,2644289,1090,29.31000,8.0,26.700,32.2000,29.2000
136355,2644289,1087,56.94000,8.0,39.400,98.1000,47.3000
136356,2644289,2033,7.01300,8.0,6.400,8.2000,6.9000
136357,2644289,1091,104.10000,8.0,82.000,131.0000,98.7500
136358,2644289,1051,68.82000,8.0,66.770,70.1400,68.7000
136359,2644289,1004,1.25600,8.0,1.060,1.5600,1.2600
136360,2644289,1095,0.56000,8.0,0.454,0.7470,0.5225


In [10]:
# Join food_nutrients with all_nutrients on nutrient_id and id 
# This is just populating more information about nutrients when looking through a food's nutrition
food_nutrients_details = food_nutrients.merge(
    all_nutrients,
    left_on='nutrient_id',
    right_on='id',
    how='left'
)

food_nutrients_details

,fdc_id,nutrient_id,amount,data_points,min,max,median,id,name,unit_name,nutrient_nbr,rank
0,2644289,1071,2.00000,8.0,2.000,2.0000,2.0000,1071,Resistant starch,G,283.0,2225.0
1,2644289,1093,172.30000,8.0,94.700,268.0000,153.5000,1093,"Sodium, Na",MG,307.0,5800.0
2,2644289,1009,12.16000,8.0,11.430,13.2300,12.0700,1009,Starch,G,209.0,2200.0
3,2644289,1090,29.31000,8.0,26.700,32.2000,29.2000,1090,"Magnesium, Mg",MG,304.0,5500.0
4,2644289,1087,56.94000,8.0,39.400,98.1000,47.3000,1087,"Calcium, Ca",MG,301.0,5300.0
5,2644289,2033,7.01300,8.0,6.400,8.2000,6.9000,2033,Total dietary fiber (AOAC 2011.25),G,293.0,1300.0
6,2644289,1091,104.10000,8.0,82.000,131.0000,98.7500,1091,"Phosphorus, P",MG,305.0,5600.0
7,2644289,1051,68.82000,8.0,66.770,70.1400,68.7000,1051,Water,G,255.0,100.0
8,2644289,1004,1.25600,8.0,1.060,1.5600,1.2600,1004,Total lipid (fat),G,204.0,800.0
9,2644289,1095,0.56000,8.0,0.454,0.7470,0.5225,1095,"Zinc, Zn",MG,309.0,5900.0


# FNDDS Excel File
Needs less preprocessing, good v1 data

In [11]:
fndds_df = pd.read_excel(FoodDBClient.fndds_foods_file)
fndds_df.head(5)

,Food code,Main food description,WWEIA Category number,WWEIA Category description,Energy (kcal),Protein (g),Carbohydrate (g),"Sugars, total\n(g)","Fiber, total dietary (g)",Total Fat (g),...,20:1\n(g),22:1\n(g),18:2\n(g),18:3\n(g),18:4\n(g),20:4\n(g),20:5 n-3\n(g),22:5 n-3\n(g),22:6 n-3\n(g),Water\n(g)
0,11100000,"Milk, NFS",1004,"Milk, reduced fat",52,3.33,4.83,4.88,0.0,2.14,...,0.002,0.0,0.074,0.008,0.0,0.003,0.000,0.001,0.0,88.92
1,11111000,"Milk, whole",1002,"Milk, whole",61,3.27,4.63,4.81,0.0,3.20,...,0.004,0.0,0.115,0.013,0.0,0.004,0.001,0.002,0.0,88.10
2,11112110,"Milk, reduced fat (2%)",1004,"Milk, reduced fat",50,3.36,4.90,4.89,0.0,1.90,...,0.002,0.0,0.061,0.007,0.0,0.003,0.000,0.001,0.0,89.10
3,11112210,"Milk, low fat (1%)",1006,"Milk, lowfat",43,3.38,5.18,4.96,0.0,0.95,...,0.001,0.0,0.033,0.004,0.0,0.001,0.000,0.000,0.0,89.70
4,11113000,"Milk, fat free (skim)",1008,"Milk, nonfat",34,3.43,4.92,5.05,0.0,0.08,...,0.000,0.0,0.005,0.000,0.0,0.000,0.000,0.000,0.0,90.80


In [12]:
print(fndds_df.columns)

Index(['Food code', 'Main food description', 'WWEIA Category number',
       'WWEIA Category description', 'Energy (kcal)', 'Protein (g)',
       'Carbohydrate (g)', 'Sugars, total\n(g)', 'Fiber, total dietary (g)',
       'Total Fat (g)', 'Fatty acids, total saturated (g)',
       'Fatty acids, total monounsaturated (g)',
       'Fatty acids, total polyunsaturated (g)', 'Cholesterol (mg)',
       'Retinol (mcg)', 'Vitamin A, RAE (mcg_RAE)', 'Carotene, alpha (mcg)',
       'Carotene, beta (mcg)', 'Cryptoxanthin, beta (mcg)', 'Lycopene (mcg)',
       'Lutein + zeaxanthin (mcg)', 'Thiamin (mg)', 'Riboflavin (mg)',
       'Niacin (mg)', 'Vitamin B-6 (mg)', 'Folic acid (mcg)',
       'Folate, food (mcg)', 'Folate, DFE (mcg_DFE)', 'Folate, total (mcg)',
       'Choline, total (mg)', 'Vitamin B-12 (mcg)',
       'Vitamin B-12, added\n(mcg)', 'Vitamin C (mg)',
       'Vitamin D (D2 + D3) (mcg)', 'Vitamin E (alpha-tocopherol) (mg)',
       'Vitamin E, added\n(mg)', 'Vitamin K (phylloquinone) (

In [26]:
# Clean column names: remove newlines, extra spaces, and replace special characters
fndds_df.columns = (
    fndds_df.columns
    .str.replace('\n', ' ', regex=True)
    .str.replace(r'\s+', '_', regex=True)
    .str.strip()
    .str.replace('(', '', regex=False)
    .str.replace(')', '', regex=False)
    .str.replace('-', '_', regex=False)
    .str.replace(',', '', regex=False)
    .str.replace('/', '_', regex=False)
    .str.lower()
)

# Aggregate omega-3 fatty acids: 18:3, 20:5 n-3, 22:5 n-3, 22:6 n-3
omega3_cols = ['18:3 g', '20:5 n-3 g', '22:5 n-3 g', '22:6 n-3 g']
# Some columns may have slightly different names after cleaning, so let's match them
matched_cols = [col for col in fndds_df.columns if any(omega in col for omega in omega3_cols)]
fndds_df['omega3_total_g'] = fndds_df[matched_cols].sum(axis=1)

# Drop columns with semicolons in their names (fatty acids)
fatty_acid_cols = [col for col in fndds_df.columns if ':' in col]
fndds_df = fndds_df.drop(columns=fatty_acid_cols)

print(fndds_df.columns, "\n")
print(f"shape: {fndds_df.shape}")
print(fndds_df.dtypes)

fndds_df.head()

Index(['food_code', 'main_food_description', 'wweia_category_number',
       'wweia_category_description', 'energy_kcal', 'protein_g',
       'carbohydrate_g', 'sugars_total_g', 'fiber_total_dietary_g',
       'total_fat_g', 'fatty_acids_total_saturated_g',
       'fatty_acids_total_monounsaturated_g',
       'fatty_acids_total_polyunsaturated_g', 'cholesterol_mg', 'retinol_mcg',
       'vitamin_a_rae_mcg_rae', 'carotene_alpha_mcg', 'carotene_beta_mcg',
       'cryptoxanthin_beta_mcg', 'lycopene_mcg', 'lutein_+_zeaxanthin_mcg',
       'thiamin_mg', 'riboflavin_mg', 'niacin_mg', 'vitamin_b_6_mg',
       'folic_acid_mcg', 'folate_food_mcg', 'folate_dfe_mcg_dfe',
       'folate_total_mcg', 'choline_total_mg', 'vitamin_b_12_mcg',
       'vitamin_b_12_added_mcg', 'vitamin_c_mg', 'vitamin_d_d2_+_d3_mcg',
       'vitamin_e_alpha_tocopherol_mg', 'vitamin_e_added_mg',
       'vitamin_k_phylloquinone_mcg', 'calcium_mg', 'phosphorus_mg',
       'magnesium_mg', 'iron_mg', 'zinc_mg', 'copper_mg', '

,food_code,main_food_description,wweia_category_number,wweia_category_description,energy_kcal,protein_g,carbohydrate_g,sugars_total_g,fiber_total_dietary_g,total_fat_g,...,zinc_mg,copper_mg,selenium_mcg,potassium_mg,sodium_mg,caffeine_mg,theobromine_mg,alcohol_g,water_g,omega3_total_g
0,11100000,"Milk, NFS",1004,"Milk, reduced fat",52,3.33,4.83,4.88,0.0,2.14,...,0.43,0.001,1.9,156,39,0,0,0.0,88.92,0.0
1,11111000,"Milk, whole",1002,"Milk, whole",61,3.27,4.63,4.81,0.0,3.20,...,0.42,0.001,1.9,150,38,0,0,0.0,88.10,0.0
2,11112110,"Milk, reduced fat (2%)",1004,"Milk, reduced fat",50,3.36,4.90,4.89,0.0,1.90,...,0.43,0.001,1.8,159,39,0,0,0.0,89.10,0.0
3,11112210,"Milk, low fat (1%)",1006,"Milk, lowfat",43,3.38,5.18,4.96,0.0,0.95,...,0.43,0.001,2.1,159,39,0,0,0.0,89.70,0.0
4,11113000,"Milk, fat free (skim)",1008,"Milk, nonfat",34,3.43,4.92,5.05,0.0,0.08,...,0.45,0.002,2.0,167,41,0,0,0.0,90.80,0.0


In [14]:
solver_df = fndds_df[[
    "food_code", "main_food_description", "energy_kcal","protein_g",
    "carbohydrate_g","sugars_total_g",
    "fiber_total_dietary_g","total_fat_g",
]]

solver_df

,food_code,main_food_description,energy_kcal,protein_g,carbohydrate_g,sugars_total_g,fiber_total_dietary_g,total_fat_g
0,11100000,"Milk, NFS",52,3.33,4.83,4.88,0.0,2.14
1,11111000,"Milk, whole",61,3.27,4.63,4.81,0.0,3.20
2,11112110,"Milk, reduced fat (2%)",50,3.36,4.90,4.89,0.0,1.90
3,11112210,"Milk, low fat (1%)",43,3.38,5.18,4.96,0.0,0.95
4,11113000,"Milk, fat free (skim)",34,3.43,4.92,5.05,0.0,0.08
...,...,...,...,...,...,...,...,...
5426,99997810,Vegetables as ingredient in curry,53,1.81,11.71,3.20,2.2,0.20
5427,99997815,Vegetables as ingredient in soups,50,1.87,10.56,2.34,2.0,0.33
5428,99997820,Vegetables as ingredient in stews,59,1.77,12.67,3.16,2.6,0.30
5429,99998130,Sauce as ingredient in hamburgers,272,1.34,17.14,13.08,0.6,22.85


In [15]:
from solver import Solver

protein_solver = Solver()
chosen, totals = protein_solver.solve(solver_df[solver_df["energy_kcal"] > 0.1], 250, 1000)

# drop any existing description column (no error if it doesn't exist)
chosen = chosen.drop(columns=['main_food_description'], errors=True)

print("-------------")
chosen = chosen.merge(right=fndds_df[["food_code","main_food_description"]], how="left", on="food_code")
chosen['total_cals'] = chosen['energy_per_g'] * chosen['amount_g']
chosen['total_protein'] = chosen['protein_per_g'] * chosen['amount_g']
print(chosen[["main_food_description", "amount_g", "total_cals", "total_protein"]])

print("-------------")

print(totals)

-------------
              main_food_description     amount_g  total_cals  total_protein
0            Fish, haddock, steamed  1000.000000  870.000000          205.9
1  Energy drink, sugar free (Vault)  1000.000000   10.000000            2.5
2       Fish, cooked, as ingredient   214.765101  176.107383           41.6
-------------
{'total_energy_kcal': 1056.1073825503358, 'total_protein_g': 250.00000000000006, 'num_items_chosen': 3}


In [25]:
# aggregate unique wweia_category_description numbers and write to csv
unique_wweia_categories = sorted(fndds_df["wweia_category_description"].unique())
wweia_categories_file = "data/fndds_wweia_food_categories.csv"
pd.DataFrame({'wweia_category_description': unique_wweia_categories}).to_csv(wweia_categories_file, index=False)
